In [ ]:
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
 
 
def poisson(rmax, ymax, xmax, maxpoints, dxy, V, howmanywires, radiusofwires, distancebetweenwires):
    """Solve the Poisson equation with the given parameters using a finite difference method."""
    # Create the space grids
    x = np.arange(-xmax, xmax, dxy)
    y = np.arange(-ymax, ymax, dxy)
    X, Y = np.meshgrid(x, y)  # Create meshgrid for 2D space
    u = np.zeros_like(X)
    r = np.sqrt((X-6)**2 + (Y+6)**2)
 
    def shape(grid, what, where, howbig, Volt):
        if what == "circle":
            r = np.sqrt((X-where[0])**2 + (Y-where[1])**2)
            grid[r < howbig[0]] = Volt  # Set values inside the circle
        elif what == "rectangle":
            x_start = int(where[0] - howbig[0] / 2)
            x_end = int(where[0] + howbig[0] / 2)
            y_start = int(-where[1] - howbig[1] / 2)
            y_end = int(-where[1] + howbig[1] / 2)
            grid[(X >= x_start) & (X <= x_end) & (Y >= y_start) & (Y <= y_end)] = Volt
 
    def bounds(grid, radiusofwires, howmanywires, distancebetweenwires):
        shape(grid, "circle", [0, 0], [radiusofwires], 0)  # Apply central circle
        for i in range(howmanywires):
            if i==0:
                continue
            # Alternate wire positions along y-axis
            positionofwires = ((-1) ** i) * math.ceil(i / 2) * (distancebetweenwires + 2 * radiusofwires)
            shape(grid, "circle", [0, positionofwires], [radiusofwires], 0)
        # Apply boundary conditions
        grid[:, 0] = -V  # Left boundary
        grid[:, -1] = -V  # Right boundary
        # Update the top and bottom boundaries using neighboring points
        grid[(0,-1), 1:-1] = 0.25*(grid[(0,-1), 2:]+grid[(0,-1), :-2]+grid[(-1,-2), 1:-1]+grid[(1,0), 1:-1])

 
    # Apply initial boundary conditions
    bounds(u, radiusofwires, howmanywires, distancebetweenwires)
    diff = 1
    # FTCS scheme for solving the Poisson equation (relaxed elliptical solver)
    for _ in range(maxpoints):
        u_new = u.copy()  # Create a copy of u to avoid overwriting values during updates
        # Vectorized 9-point stencil update
        u_new[1:-1, 1:-1] = 0.125 * (
            u[2:, 1:-1] + u[:-2, 1:-1] + u[1:-1, 2:] + u[1:-1, :-2] +
            u[2:, 2:] + u[2:, :-2] + u[:-2, 2:] + u[:-2, :-2]
        )
 
        # Convergence check: Stop if the difference is small enough
        diff = np.max(np.abs(u_new - u))
        if diff < 1e-12:
            break
        # Apply boundary conditions again after updating the grid
        bounds(u_new, radiusofwires, howmanywires, distancebetweenwires)
        u = u_new  # Update the solution
 
    return u, X, Y
 
# Parameters
rmax = 10   # Radius of internal region set to 0
ymax = 30  # Maximum y value
xmax = 10  # Maximum x value (aka l)
maxpoints = 1000  # Number of iterations
dxy = 0.2  # Grid resolution
V = 10
howmanywires = 3  # Always odd
radiusofwires = 2
distancebetweenwires = 5
 
 
def cap(points, v, a, s):
    n = len(points)
    E_tot = 0
    for i in range(0, n):
        E_tot += points[i]
    E = E_tot/n
    rho = E * 8.854*10**(-12)
    capacitance = rho/v
    cap_per_length = capacitance/(a+s)
    return cap_per_length
 
 
#xmax=np.arange(2,30,0.5)
#radiusofwires=np.arange(.1,3,0.1)
#distancebetweenwires=np.arange(1,5,0.1)
 
 
valuesforthefit=np.zeros((0, 2))
 
#for xmax in xmax:
#for radiusofwires in radiusofwires:
#for distancebetweenwires in distancebetweenwires:
    # Solve the Poisson equation
    #Solve the Poisson equation
    pot, X, Y = poisson(rmax, ymax, xmax, maxpoints, dxy, V, howmanywires, radiusofwires, distancebetweenwires)
 
 
    #Compute the gradients (electric field)
    grad_x, grad_y = np.gradient(pot, axis=(1, 0))
    elecstrength=np.sqrt(grad_x**2 + grad_y**2)
 
    middleofarray = round(len(Y) / 2)  # This would be the center of the y-axis grid.
    # Slice to get a window of points around the middle of the array (adjusted for the wire's radius and distance).
    start_idx = middleofarray - round(radiusofwires/dxy + 0.5 * distancebetweenwires/dxy )
    end_idx = middleofarray + round(radiusofwires/dxy  + 0.5 * distancebetweenwires/dxy )
    # Now select the points for capacitance calculation
    pointsforcap = elecstrength[start_idx:end_idx,0]
 
    with open(f"electricmovedby_{i}.csv", mode='w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(elecstrength)
    #valuesforthefit=np.vstack([valuesforthefit, [xmax,cap(pointsforcap,V,radiusofwires,distancebetweenwires)]])
    #valuesforthefit=np.vstack([valuesforthefit, [radiusofwires,cap(pointsforcap,V,radiusofwires,distancebetweenwires)]])
    #valuesforthefit=np.vstack([valuesforthefit, [distancebetweenwires,cap(pointsforcap,V,radiusofwires,distancebetweenwires)]])
 
def exponential_model(x, a,b,c):
    return a *(x**-b)+c
 
# Use curve_fit to fit the model to the data
params, params_covariance = curve_fit(exponential_model, valuesforthefit[:,0], valuesforthefit[:,1])
 
# Extract fitted parameters
a_fitted, b_fitted,c_fitted = params
 
# Generate the fitted curve
y_fitted = exponential_model(valuesforthefit[:,0], a_fitted, b_fitted,c_fitted)
 
print(valuesforthefit)
 
plt.figure(figsize=(8, 6))
plt.plot(valuesforthefit[:,0],valuesforthefit[:,1],marker='o', linestyle='-', color='b', label="Data")
plt.plot(valuesforthefit[:,0], y_fitted, color='red', label='Fitted curve: y = {:.2f}x + {:.2f}'.format(a_fitted, b_fitted))
plt.title("Plot of Data", fontsize=16)
plt.xlabel("X values", fontsize=12)
plt.ylabel("Y values", fontsize=12)
plt.grid(True)
plt.legend()
plt.show()
print(f"Fitted parameters: a = {a_fitted:.3e}, b = {b_fitted:.3e},c = {c_fitted:.3e}")